In [ ]:
import os
import pandas as pd
import cv2

### Dataset preparation

In [ ]:
path = r"C:\Users\chakumar\CV Projects\datasets\Racoon Images\images"

In [ ]:
imPaths = os.listdir(path)

In [ ]:
label_df = pd.read_csv(r"C:\Users\chakumar\CV Projects\datasets\train_labels_.csv")
label_df.set_index('filename',inplace = True)

In [ ]:
positive_path = r"C:\Users\chakumar\CV Projects\Object detection RCNN\racoons"
negative_path = r"C:\Users\chakumar\CV Projects\Object detection RCNN\notRacoons"

In [ ]:
def compute_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [ ]:
max_proposed_rois = 2000
MAX_POSITIVE = 30
MAX_NEGATIVE = 10
input_size = (224, 224)

total_positive_roi = 0
total_negative_roi = 0
rois_dict  = dict(columns=["filename","rois"])
pos_paths = []
pos_rois = []

for file in imPaths:
    img_file = f"{path}\{file}"
    img = cv2.imread(img_file)

    if file not in label_df.index:
        continue

    gtBoxes = []
    ground_truth_rois = label_df.loc[f"{file}"][["xmin", "ymin", "xmax", "ymax"]].values
    if ground_truth_rois.ndim == 1:
        gtBoxes.append(ground_truth_rois)
    else:
        gtBoxes = ground_truth_rois

    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()

    proposed_roi = []
    for (x, y, w, h) in rects:
        proposed_roi.append((x, y, x + w, y + h))

    positive_roi, negative_roi = 0,0

    for roi in proposed_roi[:max_proposed_rois]:
        (propStartX, propStartY, propEndX, propEndY) = roi

        for gtbox in gtBoxes:
            iou = compute_iou(roi, gtbox)
            proposed_img = None

            if iou > 0.7 and positive_roi < MAX_POSITIVE:
                proposed_img = img[propStartY:propEndY, propStartX:propEndX]
                outputpath = f"{positive_path}\{total_positive_roi}.jpg"

                positive_roi = positive_roi + 1
                total_positive_roi = total_positive_roi + 1

                pos_paths.append(outputpath)
                pos_rois.append(roi)

            overlap = propStartX >= gtbox[0]
            overlap = overlap and propStartY >= gtbox[1]
            overlap = overlap and propEndX <= gtbox[2]
            overlap = overlap and propEndY <= gtbox[3]

            if not overlap and iou < 0.05 and negative_roi < MAX_NEGATIVE:
                proposed_img = img[propStartY:propEndY, propStartX:propEndX]
                negative_roi = negative_roi + 1
                total_negative_roi = total_negative_roi + 1
                outputpath = f"{negative_path}\{total_negative_roi}.jpg"

            if proposed_img is not None:
                cv2.resize(proposed_img, input_size, interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(outputpath, proposed_img)

rois_dict['filename'] = pos_paths
rois_dict['rois'] = pos_rois

### Fine Tuning

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pickle


In [ ]:
from imutils import paths

In [ ]:
impaths = list(paths.list_images(r"C:\Users\chakumar\CV Projects\Object detection RCNN\new_dataset"))
images, labels = [], []

for impath in impaths:
    img = cv2.imread(impath)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img)
    images.append(img)
    label = impath.split(os.path.sep)[-2]
    labels.append(label)



In [ ]:
images = np.array(images, dtype = "float64")
labels = np.array(labels)

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(images, labels,
    test_size=0.20, stratify=labels, random_state=42)

In [ ]:
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                        height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")


In [ ]:
def classifer():
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
    # the base model
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(2, activation="softmax")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)
    # loop over all layers in the base model and freeze them so they will
    for layer in baseModel.layers:
        layer.trainable = False

    return model

In [ ]:
classifier_model = classifer()
classifier_model.compile(optimizer = Adam(learning_rate = 0.0001), loss = "binary_crossentropy", metrics = ["accuracy"])
model = classifier_model.fit(aug.flow(trainX, trainY, batch_size = 64),validation_data=(testX, testY), epochs = 10)

Epoch 1/10
36/36 [==============================] - 42s 1s/step - loss: 0.4358 - accuracy: 0.8335 - val_loss: 0.1399 - val_accuracy: 0.9858
Epoch 2/10
36/36 [==============================] - 42s 1s/step - loss: 0.1202 - accuracy: 0.9836 - val_loss: 0.0569 - val_accuracy: 0.9876
Epoch 3/10
36/36 [==============================] - 40s 1s/step - loss: 0.0649 - accuracy: 0.9907 - val_loss: 0.0348 - val_accuracy: 0.9912
Epoch 4/10
36/36 [==============================] - 39s 1s/step - loss: 0.0428 - accuracy: 0.9947 - val_loss: 0.0257 - val_accuracy: 0.9929
Epoch 5/10
36/36 [==============================] - 39s 1s/step - loss: 0.0343 - accuracy: 0.9938 - val_loss: 0.0209 - val_accuracy: 0.9929
Epoch 6/10
36/36 [==============================] - 40s 1s/step - loss: 0.0283 - accuracy: 0.9938 - val_loss: 0.0187 - val_accuracy: 0.9965
Epoch 7/10
36/36 [==============================] - 39s 1s/step - loss: 0.0238 - accuracy: 0.9942 - val_loss: 0.0172 - val_accuracy: 0.9965
Epoch 8/10
36/36 [==

In [ ]:
predicts = classifier_model.predict(testX)
predicts = np.argmax(predicts, axis = 1)

18/18 [==============================] - 5s 253ms/step


In [ ]:
print(classification_report(testY.argmax(axis=1), predicts,
 target_names=lb.classes_))

              precision    recall  f1-score   support

  notRacoons       1.00      0.99      1.00       320
     racoons       0.99      1.00      1.00       245

    accuracy                           1.00       565
   macro avg       1.00      1.00      1.00       565
weighted avg       1.00      1.00      1.00       565



In [ ]:
classifier_model.save(r"C:\Users\chakumar\CV Projects\Object detection RCNN\classifiermodel", save_format="h5")
# serialize the label encoder to disk

f = open(r"C:\Users\chakumar\CV Projects\Object detection RCNN\bl", "wb")
f.write(pickle.dumps(lb))
f.close()

### Inferencing

In [ ]:
from imutils.object_detection import non_max_suppression
from tensorflow.keras.models import load_model

In [ ]:
model  =  load_model(r"C:\Users\chakumar\CV Projects\Object detection RCNN\classifiermodel")
lb = pickle.loads(open(r"C:\Users\chakumar\CV Projects\Object detection RCNN\bl", "rb").read())

In [ ]:
raccon1 = cv2.imread(r"C:\Users\chakumar\CV Projects\datasets\Racoon Images\images\raccoon-1.jpg")

ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(raccon1)
ss.switchToSelectiveSearchFast()
rects = ss.process()

proposals = []
boxes = []
for (x,y,w,h) in rects:
    roi = raccon1[y:y+h, x:x+w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (224,224))

    roi = preprocess_input(roi)
    proposals.append(roi)
    boxes.append((x,y,x+w,y+h))

proposals = np.array(proposals, dtype = 'float64')
boxes = np.array(boxes, dtype = "int32")

predictions = model.predict(proposals)



45/45 [==============================] - 12s 272ms/step


In [ ]:
roi_labels = lb.classes_[np.argmax(predictions, axis = 1)]
racoons_idx = np.where(roi_labels == "racoons")[0]

In [ ]:
racoons_boxes = boxes[racoons_idx]
racoons_probs = predictions[racoons_idx][:,-1]

min_porb = 0.9999
racoons_idx_valid = np.where(racoons_probs > min_porb)[0]
racoons_boxes = racoons_boxes[racoons_idx_valid]
racoons_probs = racoons_probs[racoons_idx_valid]

In [ ]:
nms_img = raccon1.copy()
nmsbox = non_max_suppression(racoons_boxes, racoons_probs)
for idx in range(nmsbox.shape[0]):
    Xin,Yin,Xend,Yend = nmsbox[idx]

    cv2.rectangle(nms_img, (Xin, Yin), (Xend, Yend),(0, 255, 0), 2)

    text= f"Raccoon:{idx}"

    cv2.putText(nms_img, text, (Xin, Yin),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

cv2.imwrite("nms_img.jpg", nms_img)

True